In [60]:
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer as simp
from sklearn.model_selection import train_test_split as tts
from sklearn import tree

In [61]:
boston_data = pd.read_csv(r'C:\Users\Sean\Desktop\EECE_FINAL\fy2023-property-assessment-data.csv')
boston_data.isna().sum()

PID                        0
CM_ID                  89467
GIS_ID                     0
ST_NUM                  9531
ST_NAME                    0
UNIT_NUM               99955
CITY                       5
ZIP_CODE                   3
BLDG_SEQ                   1
NUM_BLDGS                  0
LUC                        0
LU                         0
LU_DESC                    0
BLDG_TYPE               2526
OWN_OCC                    0
OWNER                      0
OWNER MAIL ADDRESS         1
RES_FLOOR              33505
CD_FLOOR              110071
RES_UNITS             170297
COM_UNITS             180004
RC_UNITS              180590
LAND_SF                 7545
GROSS_AREA             34633
LIVING_AREA            34923
LAND_VALUE                 0
BLDG_VALUE                 0
TOTAL_VALUE                0
GROSS_TAX                  0
YR_BUILT               22930
YR_REMODEL             95227
STRUCTURE_CLASS       163567
ROOF_STRUCTURE         36420
ROOF_COVER             36418
INT_WALL      

In [62]:
#This is the create classes for dec_tree
#land_value+building_value=total_value
#gross_tax is also really easy to get total_value
b_max = boston_data['TOTAL_VALUE'].max()
print(b_max)
segments = pd.IntervalIndex.from_tuples([(0,333333),(333334,666666),(666667,999999),(1000000,5000000),(5000001,b_max+1)])
boston_data['segment_cost'] = pd.cut(boston_data['TOTAL_VALUE'], [0,350000,550000,700000,1000000,b_max+1], labels=['a','b','c','d','e'], ordered = False)
boston_data['segment_cost'].head()
boston_data.segment_cost.value_counts()

2143052600


e    38054
b    37118
d    35532
a    30465
c    28910
Name: segment_cost, dtype: int64

In [63]:
#Preprocessing features
le = preprocessing.LabelEncoder()
boston_data['ROOF_STRUCTURE'] = le.fit_transform(boston_data['ROOF_STRUCTURE'])
boston_data['ROOF_COVER'] = le.fit_transform(boston_data['ROOF_COVER'])
boston_data['INT_WALL'] = le.fit_transform(boston_data['INT_WALL'])
boston_data['EXT_FNISHED'] = le.fit_transform(boston_data['EXT_FNISHED'])
boston_data['INT_COND'] = le.fit_transform(boston_data['INT_COND'])
boston_data['EXT_COND'] = le.fit_transform(boston_data['EXT_COND'])
boston_data['OVERALL_COND'] = le.fit_transform(boston_data['OVERALL_COND'])
boston_data['ST_NAME'] = le.fit_transform(boston_data['ST_NAME'])
boston_data['CITY'] = le.fit_transform(boston_data['CITY'])
boston_data['OWNER MAIL ADDRESS'] = le.fit_transform(boston_data['OWNER MAIL ADDRESS'])
boston_data['LU'] = le.fit_transform(boston_data['LU'])
boston_data['LU_DESC'] = le.fit_transform(boston_data['LU_DESC'])
boston_data['BLDG_TYPE'] = le.fit_transform(boston_data['BLDG_TYPE'])
boston_data['OWN_OCC'] = le.fit_transform(boston_data['OWN_OCC'])
boston_data['KITCHEN_TYPE'] = le.fit_transform(boston_data['KITCHEN_TYPE'])


boston_data['ROOF_STRUCTURE'] = boston_data['ROOF_STRUCTURE'].fillna(0)
boston_data['ROOF_COVER'] = boston_data['ROOF_COVER'].fillna(0)
boston_data['INT_WALL'] = boston_data['INT_WALL'].fillna(0)
boston_data['EXT_FNISHED'] = boston_data['EXT_FNISHED'].fillna(0)
boston_data['INT_COND'] = boston_data['INT_COND'].fillna(0)
boston_data['EXT_COND'] = boston_data['EXT_COND'].fillna(0)
boston_data['OVERALL_COND'] = boston_data['OVERALL_COND'].fillna(0)
boston_data['FIREPLACES'] = boston_data['FIREPLACES'].fillna(0)
boston_data['NUM_PARKING'] = boston_data['NUM_PARKING'].fillna(0)
boston_data['BED_RMS'] = boston_data['BED_RMS'].fillna(0)
boston_data['FULLBTH'] = boston_data['FULL_BTH'].fillna(0)
boston_data['HLF_BTH'] = boston_data['HLF_BTH'].fillna(0)
boston_data['KITCHENS'] = boston_data['KITCHENS'].fillna(0)
boston_data['TT_RMS'] = boston_data['TT_RMS'].fillna(0)
boston_data['YR_BUILT'] = boston_data['YR_BUILT'].fillna(0)
boston_data['YR_REMODEL'] = boston_data['YR_REMODEL'].fillna(0)
boston_data['ZIP_CODE'] = boston_data['ZIP_CODE'].fillna(0)
boston_data['BLDG_SEQ'] = boston_data['BLDG_SEQ'].fillna(0)
boston_data['CITY'] = boston_data['CITY'].fillna(0)
boston_data['OWNER MAIL ADDRESS'] = boston_data['OWNER MAIL ADDRESS'].fillna(0)
boston_data['KITCHEN_TYPE'] = boston_data['KITCHEN_TYPE'].fillna(0)
boston_data['FULL_BTH'] = boston_data['FULL_BTH'].fillna(0)

b = boston_data['LAND_SF'].mean()
b1 = boston_data['GROSS_AREA'].mean()
b2 = boston_data['LIVING_AREA'].mean()
boston_data['LAND_SF'] = boston_data['LAND_SF'].fillna(b)
boston_data['GROSS_AREA'] = boston_data['GROSS_AREA'].fillna(b1)
boston_data['LIVING_AREA'] = boston_data['LIVING_AREA'].fillna(b2)

#THIS IS SKETCHY BUT MISSING Y VALUES
#NEED TO FIGURE OUT WHY MY BUCKETING MISSED 10548 POINTS
boston_data['segment_cost'] = boston_data['segment_cost'].fillna('a')

In [64]:
#Multiclass dec tree construction
X = boston_data[boston_data.columns.drop(['segment_cost', 'OWNER','CM_ID', 'GIS_ID', 'LAND_VALUE', 'TOTAL_VALUE','BLDG_VALUE','GROSS_TAX','ST_NUM','UNIT_NUM','RES_FLOOR',
'CD_FLOOR','RES_UNITS','COM_UNITS','RC_UNITS','STRUCTURE_CLASS','BDRM_COND','BTHRM_STYLE1','BTHRM_STYLE2','BTHRM_STYLE3','KITCHEN_STYLE1','KITCHEN_STYLE2',
'KITCHEN_STYLE3', 'HEAT_TYPE','HEAT_FUEL','AC_TYPE','ORIENTATION','PROP_VIEW','CORNER_UNIT'])]

Y = boston_data['segment_cost']

In [65]:
Y.isna().sum()

0

In [66]:
X_train, X_test, y_train, y_test = tts(X,Y,test_size=.25,random_state=42)
clf = tree.DecisionTreeClassifier(max_depth=12)
clf1 = clf.fit(X_train, y_train)
print('Accuracy on Test Set: ', clf1.score(X_test, y_test))

Accuracy on Test Set:  0.7652412693491596
